In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor

from statistics import mean
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval #for hyperparameter tuning and minimizing

from cyclic_boosting.pipelines import pipeline_CBClassifier

from datetime import date
from datetime import datetime
from datetime import timedelta

import tensorflow as tf
import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from keras.layers import LSTM

import itertools

from sklearn.discriminant_analysis import StandardScaler


import seaborn as sns 

from scipy.stats import boxcox 
from scipy.special import inv_boxcox

from termcolor import colored

import joblib
from tqdm import tqdm 

import swifter
from pandarallel import pandarallel

In [2]:
pandarallel.initialize()
modelDict = joblib.load('modelDict.pkl')


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
samples = pd.read_csv('preprocessed_lstm.csv')
building_id = pd.read_csv('devices.csv')
test = pd.read_csv('test.csv', header = None)
reading_types = pd.read_csv('reading_types.csv')

predictions = test

In [4]:
predictions.columns = ['device_id', 'date', 'value_type_id']
samples['date'] = pd.to_datetime(samples['date'])
predictions = pd.merge(predictions, building_id, on='device_id', how='inner')
predictions['date'] =  pd.to_datetime(predictions['date'])
predictions['floored_date'] = predictions['date'].dt.floor('H')

In [5]:
predictions.info() 
progressBar = tqdm(total = len(predictions))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048576 entries, 0 to 1048575
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   device_id      1048576 non-null  int64         
 1   date           1048576 non-null  datetime64[ns]
 2   value_type_id  1048576 non-null  int64         
 3   building_id    1048576 non-null  int64         
 4   floored_date   1048576 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(3)
memory usage: 40.0 MB


  0%|          | 0/1048576 [00:00<?, ?it/s]

In [6]:
def get_predictions(row):
    type = row['value_type_id']
    date = row['floored_date']
    building = row['building_id']
    model = modelDict[building][type - 1]

    start_date = date  - timedelta(hours = 1)
    inputs = samples[(samples['building_id'] == building)]
    inputs = inputs[(inputs['date'] <= start_date)].head(1)


    inputs = inputs.drop(['Unnamed: 0', 'building_id', 'date'], axis = 1)  

    inputs = inputs[[str(type)]]

    scaler = StandardScaler() 
    scaler = scaler.fit(inputs)
    inputs_scaled = scaler.transform(inputs)

    prediction = model.predict(inputs_scaled, verbose = 0)

    pred = scaler.inverse_transform(prediction)[0, 0]

    progressBar.update(1)

    return pred #prob have to grab the value or smt





In [8]:
predictions['value'] = predictions.parallel_apply(get_predictions, axis=1)

NameError: name 'modelDict' is not defined

In [ ]:
predictions = predictions.drop(['floored_date', 'building_id'], axis = 1)
predictions.to_csv('predictions.csv', header = False)